In [1]:
import dagshub
import mlflow
mlflow.set_tracking_uri("https://dagshub.com/aryan0147/Capstone-Project-2.mlflow")

dagshub.init(repo_owner='aryan0147', repo_name='Capstone-Project-2', mlflow=True)

Accessing as aryan0147

Initialized MLflow to track repo "aryan0147/Capstone-Project-2"

Repository aryan0147/Capstone-Project-2 initialized!

In [2]:
# Set or create an experiment
mlflow.set_experiment("ML-Algos")

<Experiment: artifact_location='mlflow-artifacts:/d43192cc72bb48deb03d07938721755b', creation_time=1739197271560, experiment_id='12', last_update_time=1739197271560, lifecycle_stage='active', name='ML-Algos', tags={}>

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna


In [4]:
df = pd.read_csv('../reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [6]:
import mlflow
import optuna
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 2: TF-IDF vectorization setup
ngram_range = (1, 3)  # Trigram
max_features = 10000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

# **Step 3: Train-test split BEFORE any class balancing (Prevents Data Leakage)**
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category']
)

# **Step 4: Fit TF-IDF only on training data**
X_train_vec = vectorizer.fit_transform(X_train)  # Fit only on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data without refitting

# **Step 5: Compute Valid Class Priors Instead of SMOTE**
unique_classes, class_counts = np.unique(y_train, return_counts=True)  # Ensure no negative values
class_priors = class_counts / class_counts.sum()  # Convert counts into probabilities (sum to 1)
class_prior_dict = dict(zip(unique_classes, class_priors))  # Dictionary mapping for reference

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_ClassWeights_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

# Step 6: Optuna objective function for Multinomial Naive Bayes
def objective_mnb(trial):
    alpha = trial.suggest_float('alpha', 1e-4, 1.0, log=True)  # Tuning the smoothing parameter

    # MultinomialNB model setup with computed class priors
    model = MultinomialNB(alpha=alpha, fit_prior=True)  # ✅ `fit_prior=True` automatically learns priors

    # Fit and evaluate the model
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    return accuracy_score(y_test, y_pred)

# Step 7: Run Optuna for Multinomial Naive Bayes, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_mnb, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = MultinomialNB(alpha=best_params['alpha'], fit_prior=True)  # ✅ No manual `class_prior`

    # Log the best model with MLflow
    log_mlflow("MultinomialNB", best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the experiment for Multinomial Naive Bayes
run_optuna_experiment()


[I 2025-02-10 20:20:23,109] A new study created in memory with name: no-name-03cbdaea-f8f3-4e5f-99ad-619410fcd0ad
[I 2025-02-10 20:20:23,124] Trial 0 finished with value: 0.6702577389881358 and parameters: {'alpha': 0.010568558316591196}. Best is trial 0 with value: 0.6702577389881358.
[I 2025-02-10 20:20:23,131] Trial 1 finished with value: 0.6710759579980908 and parameters: {'alpha': 0.0004630880800846927}. Best is trial 1 with value: 0.6710759579980908.
[I 2025-02-10 20:20:23,143] Trial 2 finished with value: 0.6706668484931133 and parameters: {'alpha': 0.007634314493688671}. Best is trial 1 with value: 0.6710759579980908.
[I 2025-02-10 20:20:23,155] Trial 3 finished with value: 0.6652120550934133 and parameters: {'alpha': 0.22196072863689062}. Best is trial 1 with value: 0.6710759579980908.
[I 2025-02-10 20:20:23,166] Trial 4 finished with value: 0.6668484931133233 and parameters: {'alpha': 0.17978580291215276}. Best is trial 1 with value: 0.6710759579980908.
[I 2025-02-10 20:20:23

🏃 View run MultinomialNB_ClassWeights_TFIDF_Trigrams at: https://dagshub.com/aryan0147/Capstone-Project-2.mlflow/#/experiments/12/runs/578fcd3cca4b4026a99f5d88e547762c
🧪 View experiment at: https://dagshub.com/aryan0147/Capstone-Project-2.mlflow/#/experiments/12
